In [1]:
import re
import json
import nltk
import pandas as pd
from collections import Counter,OrderedDict
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# import all necessary dependencies for training
from sklearn import metrics
from sklearn.metrics import jaccard_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
import warnings
warnings.filterwarnings('always') 

In [2]:
human_tags_data = pd.read_csv('all_annotated_tickets_clean.csv')
tags_data = pd.read_csv('crowdtruth.csv')
candidate_tags = tags_data['tags'].to_numpy()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,24

In [3]:
human_tags_data.head()

,id,title,content,0,1,2,3,4,5,6,...,AW8JVJQNLTXND,AXFC96MD712VJ,AY00EYSHCVC1N,AY9JW8EIKVBR9,AYIFHDQSXQJ6B,AZBO9PRZKUY5G,AZIPDRVJSCSNB,AZL5V53B7C7L6,lang,title_content
0,808689,adblock plus filter subscription subscription ...,update adblock plus recently late hour late da...,addon|firefox-3616|desktop|firefox-36|windows-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,adblock plus filter subscription subscription ...
1,808690,rid cookie scorecardresearch.com appear list c...,permanently remove scorecardresearch.com uid c...,data|firefox-3616|desktop|firefox-36|windows-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,rid cookie scorecardresearch.com appear list c...
2,808691,firefox 4 open google chat work google,use google chat late download firefox google c...,firefox-40|websites|desktop|windows-vista,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,firefox 4 open google chat work google use goo...
3,808692,history last max 3 day plugin dissaprear close...,wish ff feature punch people internet hard!i'v...,firefox-40|data|desktop|windows-7,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,history last max 3 day plugin dissaprear close...
4,808693,number verification image ticket site display,try buy ticket line tickets.com ticketmaster e...,firefox-36|websites|firefox-3616|desktop|windo...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en,number verification image ticket site display ...


In [4]:
frame = { 'ticket_id': human_tags_data['id'], 'title_content': human_tags_data['title_content'] } 
dataset = pd.DataFrame(frame) 

In [5]:
dataset.head()

,ticket_id,title_content
0,808689,adblock plus filter subscription subscription ...
1,808690,rid cookie scorecardresearch.com appear list c...
2,808691,firefox 4 open google chat work google use goo...
3,808692,history last max 3 day plugin dissaprear close...
4,808693,number verification image ticket site display ...


In [6]:
for tag in candidate_tags:
    labels = [1 if tag in str(item) else 0 for item in human_tags_data['0']]
    all_zero = [0]*human_tags_data.index.size
    ### find all tags that not matched with any ticket
    if all_zero == labels:
        print(tag)
    dataset[tag] = labels

portal
hyperlinks
pictures
resize
hacked
purchase
checkbox
empty
fails
stuck
transfer
engines
format
boxes
source
macbook
enhanced
safety
quantum
payment
unusable
context
previous
exception
prevent
opera
applications
crashed
changed
setup
library
pop-up
stopped
re-install
scripts
external
failed
random
gone
can't
won't
response
favorites
logged
function
laptop
program
without
domain
doesn't
win7
downloaded
reboot
manually
failure
explorer
printers
unable
retrieve
win10
information
opening
allow
rendering


In [7]:
dataset.head()

,ticket_id,title_content,bing,widget,pinterest,instagram,console,pdf,paypal,wikipedia,...,opening,allow,work,rendering,webpage,updated,apple,websites,opens,downloads
0,808689,adblock plus filter subscription subscription ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,808690,rid cookie scorecardresearch.com appear list c...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,808691,firefox 4 open google chat work google use goo...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,808692,history last max 3 day plugin dissaprear close...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,808693,number verification image ticket site display ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
train, test = train_test_split(dataset, random_state=42, test_size=0.30, shuffle=True)

In [9]:
train.index.size,test.index.size

(223020, 95581)

In [10]:
train_text = train['title_content']
test_text = test['title_content']

In [11]:
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', norm='l2')
vectorizer.fit(train_text)
vectorizer.fit(test_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents='unicode',
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [12]:
x_train = vectorizer.transform(train_text)
x_test = vectorizer.transform(test_text)

In [13]:
y_train = train.drop(labels = ['ticket_id','title_content'], axis=1)
y_test = test.drop(labels = ['ticket_id','title_content'], axis=1)

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
Naive_pipeline = Pipeline([('clf', ComplementNB())])

total_score = 0
for tag in candidate_tags:
    classifier = Naive_pipeline.fit(x_train, y_train[tag])
    prediction = (classifier.predict_proba(x_test)[:,1] >= 0.35).astype(bool)
    score = jaccard_score(prediction,y_test[tag])
    total_score += score
print(total_score)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 due to no true or predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

5.445181845530874
